In [1]:
from datasets import load_from_disk
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from datasets import load_from_disk
import eval
import torch

2025-04-07 03:42:34.242312: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-07 03:42:34.244485: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-07 03:42:34.248515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744015354.256326 3030865 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744015354.258557 3030865 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regist

In [2]:
model_id = "mistralai/Mistral-7B-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
adapter_path = "weights/gsm8k_magicoder_avg/gsm8k_magicoder_avg"

merged_adapter_name = "gsm8k_magicoder_avg"
model.load_adapter(adapter_path, adapter_name=merged_adapter_name)
model.set_adapter(merged_adapter_name)

This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



In [4]:
k = 5
max_new_tokens = 256
temperature = 0.2
candidate_file = "data/generated_results/merged_candidates.json"
tokenizer = AutoTokenizer.from_pretrained(model_id)
dataset = load_from_disk("data/humaneval_dataset")["test"]

In [ ]:
eval.generate_candidates(
    model=model,
    tokenizer=tokenizer,
    dataset=dataset,
    k=k,
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    save_path=candidate_file,
)

Generating candidates:   0%|          | 0/164 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [5]:
candidate_file = "data/generated_results/merged_candidates.json"
candidates = eval.load_candidates(candidate_file)
eval.evaluate_candidates(candidates, k=k)

Candidates loaded from data/generated_results/merged_candidates.json
Evaluation results:
Pass@1: 84.63%
Pass@5: 97.56%
